# The Forward Mapping Between Articulation and Acoustics

2018-08-20 Jaekoo Kang

## Refs:
- Slider: https://plot.ly/~jackluo/2228/moving-sma-slider-visualization-with-plo/#/
- Curve fitting: https://plot.ly/~rreusser/90/chebyshev-points-for-least-squares-curve/#/
- Subplots: https://plot.ly/python/subplots/
- Mixed subplots: https://plot.ly/python/mixed-subplots/
- Gap plot: https://plot.ly/python/line-charts/

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import display, HTML
init_notebook_mode(connected=True)

import time

In [2]:
# Load single speaker data
df = pd.read_pickle('JW12.pckl')

In [3]:
df.head()

,Subj,FName,PreLab,NexLab,Label,Context,Dur,T1x,T1y,T2x,...,segEnd,GONS,NONS,MAXC,NOFFS,GOFFS,GesTarDur,timeMidWithinGest,timeMid2GestDiff,GestFound
0,JW12,JW12_TP007,P,N,AE1,P-AE1-N,0.119728,-15.086,-1.4980,-29.573500,...,5.962424,5.801770,5.884162,5.904760,5.918492,5.966554,0.034330,1.0,0.000000,1
1,JW12,JW12_TP010,HH,D,AE1,HH-AE1-D,0.099773,-17.583,3.0545,-30.327500,...,6.339805,6.076410,6.186266,6.206864,6.241194,6.378514,0.054928,0.0,0.048724,1
2,JW12,JW12_TP017,N,S,AE1,N-AE1-S,0.119728,-10.927,-0.0570,-25.707001,...,1.635277,1.380066,1.510520,1.592912,1.613510,1.682170,0.102990,1.0,0.000000,1
3,JW12,JW12_TP017,K,R,AE1,K-AE1-R,0.079819,-22.037,12.3780,-33.766001,...,2.283803,2.100996,2.300110,2.320708,2.348172,2.444296,0.048062,0.0,0.056216,1
4,JW12,JW12_TP017,IY0,N,AE1,IY0-AE1-N,0.049887,-15.916,7.5980,-29.587999,...,2.583123,2.492358,2.602214,2.615946,2.636544,2.705204,0.034330,0.0,0.044035,1


In [4]:
# Load pal, pha
with open('pal_pha.pckl', 'rb') as pckl:
    pal, pha = pickle.load(pckl)
# Set parameters
artic_col = ['T1x', 'T1y', 'T2x', 'T2y', 'T3x', 'T3y',
             'T4x', 'T4y', 'ULx', 'ULy', 'LLx', 'LLy', 'MNIx', 'MNIy']
acous_col = ['F1', 'F2']
# vowel_list = ['AE1', 'AH1', 'AO1', 'EH1', 'IH1', 'AA1', 'IY1', 'UW1', 'UH1']
vowel_list = ['AE1', 'AA1', 'IY1', 'UW1']

# Prepare dataset
X_raw = df.loc[:, artic_col].copy().as_matrix()
Y_raw = df.loc[:, acous_col].copy().as_matrix()
# Standardize before PCA
# Articulation
X_scaler = StandardScaler().fit(X_raw)
X_std = X_scaler.transform(X_raw)  # cf .inverse_transform()
# Acoustics
Y_scaler = StandardScaler().fit(Y_raw)
Y_std = Y_scaler.transform(Y_raw)

# PCA
pca = PCA(n_components=3)
pca.fit(X_std)
X_reduced = pca.transform(X_std)

# Linear Regression
#  X*w = y
W = np.dot(np.linalg.pinv(X_reduced), Y_std)

# Compute median articulation & acoustic values
medianArtic = np.zeros((len(vowel_list), 14))
medianAcous = np.zeros((len(vowel_list), 2))
for i, v in enumerate(vowel_list):
    x = df.loc[df.Label == v, artic_col].as_matrix()
    y = df.loc[df.Label == v, acous_col].as_matrix()
    medianArtic[i, :] = np.median(x, axis=0)  # 7x14
    medianAcous[i, :] = np.median(y, axis=0)  # 7x2

# Estimate F1, F2 for each vowel
y_scaled_vowels = np.dot(pca.transform(X_scaler.transform(medianArtic)), W)
y_vowels = Y_scaler.inverse_transform(y_scaled_vowels)  # 7x2

## Make template

In [5]:
# PCA space
trace1 = go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    marker=dict(
        size=5,
        symbol='circle',
        color='rgb(0,0,217)',
        opacity=0.8
    ),
)

# Articulator space
# pharynx
trace2 = go.Scatter(
    x=pha[:,0],
    y=pha[:,1],
    mode='lines',
    name='Pharynx',
    xaxis='x2',
    yaxis='y2',
    line=dict(
        color=('rgb(0,0,0)')
    ),
    hoverinfo='none',
)
# palate
trace3 = go.Scatter(
    x=pal[:,0],
    y=pal[:,1],
    mode='lines',
    name='Palate',
    xaxis='x2',
    yaxis='y2',
    line=dict(
        color=('rgb(0,0,0)')
    ),
    hoverinfo='none',
)
# Reference formants
trace4 = go.Scatter(
    x=y_vowels[:,[0]],
    y=y_vowels[:,[1]],
    name='Formant',
    mode='markers',
    xaxis='x3',
    yaxis='y3',
    marker=dict(
        size=3,
        symbol='circle',
        color='rgb(0,0,217)',
        opacity=0.8
    ),
)

## Make data

In [6]:
data = [trace1,trace2,trace3,trace4]

smin = -5
smax = 5
inc = 0.1
pc_step = [float(f'{i:.1f}') for i in np.arange(smin, smax+inc, inc)]

for s in pc_step:
    pc_vec = np.array([[s,0,0]])
    
    artics = X_scaler.inverse_transform(pca.inverse_transform(pc_vec))
    formants = Y_scaler.inverse_transform(np.dot(pc_vec, W))
    
    T1x, T1y, T2x, T2y, T3x, T3y, T4x, T4y, ULx, ULy, LLx, LLy, JAWx, JAWy = artics[0]
    F1, F2 = formants[:,0], formants[:,1]
    
    # PCs
    P = go.Scatter3d(
        x=[s],
        y=[0],
        z=[0],
        mode='markers',
        visible=False,
        marker=dict(
            size=5,
            symbol='circle',
            color='rgb(0,0,217)',
            opacity=0.8
        ),
    )
    # Articulators
    A = go.Scatter(
        x=[T1x,T2x,T3x,T4x,None,ULx,None,LLx,None,JAWx],
        y=[T1y,T2y,T3y,T4y,None,ULy,None,LLy,None,JAWy],
        name='',
        visible=False,
        mode='lines+markers',
        xaxis='x2',
        yaxis='y2',
        line=dict(
            shape='spline',
        ),
        marker=dict(
            size=5,
            color='rgb(0,0,0)',
        ),
    )
    # Formants
    F = go.Scatter(
        x=F2,
        y=F1,
        name='',
        visible=False,
        mode='markers',
        xaxis='x3',
        yaxis='y3',
        marker = dict(
            size=5,
            color='rgb(0,0,0)',
        ),
    )
    
    data.append(P)
    data.append(A)
    data.append(F)

## Make sliders

In [7]:
sliders = dict(
    # General
    steps=[],
    active=pc_step.index(0),
    currentvalue=dict(
        font=dict(size=10,color='rgb(0,0,0)'),
        prefix='',
        xanchor='center',
        offset=3,
    ),
    
    # Placement
    x=0.01,
    y=0.9,
    len=0.25,
    pad=dict(t=1,b=1),
    xanchor='left',
    yanchor='bottom',
    tickcolor='rgba(255,255,255,1)',
    font=dict(color='rgba(255,255,255,1)'),
)

# For data
for i, s in enumerate(pc_step):
    step = dict(
        method='restyle',
        label=str(s),
        value=str(s),
        args=['visible', [False,True,True,True]+[False]*len(pc_step)*3],
    )
    
    step['args'][1][i*3+4] = True # Selected value for PC space through slider
    step['args'][1][i*3+5] = True # Selected value for articulator space through slider
    step['args'][1][i*3+6] = True # Selected value for formant space through slider
    sliders['steps'].append(step)

## Make layout

In [8]:
layout = go.Layout(
    # General
    title='Articulation to Acoustics',
    height=400,
    margin=dict(
    l=30,
    r=30,
    b=70,
    t=70),
    showlegend=False,
    
    # Slider
    sliders=[sliders],
    
    # Axis
    scene = dict(
        xaxis=dict(
            title='PC1',
            range=[-5,5],
            dtick=1,
            tickfont=dict(
                size=10,
            ),
        ),
        yaxis=dict(
            title='PC2',
            range=[-5,5],
            dtick=1,
            tickfont=dict(
                size=10,
            ),
        ),
        zaxis=dict(
            title='PC3',
            range=[-5,5],
            dtick=1,
            tickfont=dict(
                size=10,
            ),
        ),
        domain=dict(
            x=[0,0.26],
            y=[0,0.9],
        ),
        camera=dict(
            eye=dict(x=2.5,y=2.5,z=0.8)
        ),
    ),
    xaxis2=dict(
        linecolor='black',
        mirror=True,
        title='Midsagittal view',
        domain=[0.36,0.62],
        range=[-90,40],
        dtick=20,
        tickangle=0,
        zeroline=False,
        tickfont=dict(
            size=10,
        ),
        fixedrange=True,
    ),
    yaxis2=dict(
        linecolor='black',
        mirror=True,
        domain=[0,0.75],
        range=[-30,30],
        dtick=10,
        anchor='x2',
        zeroline=False,
        tickfont=dict(
            size=10,
        ),
        fixedrange=True,
    ),
    xaxis3=dict(
        title='Formant space',
        linecolor='black',
        mirror=True,
        domain=[0.72,0.98],
        range=[2200,800],
        dtick=250,
        zeroline=False,
        tickfont=dict(
            size=10,
        ),
        fixedrange=True,
    ),
    yaxis3=dict(
        linecolor='black',
        mirror=True,
        domain=[0,0.75],
        anchor='x3',
        range=[800,200],
        dtick=100,
        zeroline=False,
        tickfont=dict(
            size=10,
        ),
        fixedrange=True,
    ),
)

In [9]:
annotations = [
    # PCA xlabel
    dict(
        text='PCA space',
        showarrow=False,
        xref='paper',
        yref='paper',
        x=0.1,
        y=-0.2,
        font=dict(
            size=14
        ),
    ),
]

layout['annotations'] = annotations

In [10]:
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='forward_mapping')